In [3]:
import pandas as pd
import os
from pathlib import Path

# Define o caminho da pasta
logs_path = Path('logs_em_tratamento')

# Lista todos os arquivos de bloco concatenados
bloco_files = list(logs_path.glob('bloco_*_concatenado.csv'))

# Cria uma lista para armazenar os DataFrames
dfs = []

# Lê cada arquivo mantendo a estrutura original
for file in bloco_files:
    df = pd.read_csv(file, sep='\t')  # Remove a definição manual de colunas
    dfs.append(df)

# Concatena todos os DataFrames
logs_totais_tratados = pd.concat(dfs, ignore_index=True)

# Remove duplicatas se houver
#logs_totais_tratados = logs_totais_tratados.drop_duplicates()

# Salva o DataFrame consolidado mantendo o formato original
output_file = logs_path / 'logs_totais_tratados.csv'
logs_totais_tratados.to_csv(output_file, sep='\t', index=False)  # Adiciona sep='\t'

print(f"Arquivo criado com sucesso em: {output_file}")
print(f"Total de registros: {len(logs_totais_tratados)}")

Arquivo criado com sucesso em: logs_em_tratamento\logs_totais_tratados.csv
Total de registros: 13485
